<center> <b><h1>CHESS LENS</h1></b><br>
<b><h2>Team BSB64</h2></b></center>

In [1]:
import sys
sys.path.append('./utils')

import os

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import occupancy_detector as od
import chessboard_processor as cbd
import detector as md
import visualizer

In [4]:
board_matrix = [
    ["r", "n", "b", "q", "k", "b", "n", "r"],
    ["p"] * 8,
    [""] * 8,
    [""] * 8,
    [""] * 8,
    [""] * 8,
    ["P"] * 8,
    ["R", "N", "B", "Q", "K", "B", "N", "R"]
]

## Preprocess the Chessboard

In [5]:
cbd.process_chessboard("images/board/test3.jpeg","images/demo5/","box",True)

QSocketNotifier: Can only be used with threads started with QThread


## Checking the ACCURACY of OCCUPANCY Detector

In [40]:
### THIS CELL IS REDUNDANT, DO NOT RUN IT!!! ###


folder = "images/demo4"
actual_labels, predicted_labels = [],[]

print("Status\t\tActual\tPredicted")
for filename in os.listdir(folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        file_path = os.path.join(folder, filename)
        actual_labels+=[1] if "true" in filename else [0]

        box = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        pred = od.is_occupied(box)
        predicted_labels+=[1] if pred==True else [0]
        print(f"Processing:{file_path}\t\t{actual_labels[-1]}\t{predicted_labels[-1]}")
        

Status		Actual	Predicted
Std_dev = 54.40071551481788
edge_density = 0.1127340456817959
Processing:images/demo4/box-3-true.jpg		1	1
Std_dev = 52.374225152294635
edge_density = 0.05989608861876287
Processing:images/demo4/box-11-true.jpg		1	1
Std_dev = 30.213455232881326
edge_density = 0.019507891383197726
Processing:images/demo4/box-46.jpg		0	0
Std_dev = 36.0466682494598
edge_density = 0.055680815606313105
Processing:images/demo4/box-61-true.jpg		1	1
Std_dev = 34.94339152115216
edge_density = 0.026958141358690325
Processing:images/demo4/box-31.jpg		0	1
Std_dev = 37.36754620318535
edge_density = 0.021860601901774336
Processing:images/demo4/box-21.jpg		0	1
Std_dev = 35.35293974218647
edge_density = 0.02333104597588472
Processing:images/demo4/box-20.jpg		0	1
Std_dev = 38.48196666958099
edge_density = 0.02627193412410548
Processing:images/demo4/box-16.jpg		0	1
Std_dev = 31.157530912803576
edge_density = 0.021468483482011566
Processing:images/demo4/box-29.jpg		0	1
Std_dev = 38.20442452930324


In [8]:
import os
import cv2
import numpy as np
from sklearn.metrics import accuracy_score

# === Rule-based occupancy detector ===
def is_occupied(img, std_thresh, edge_thresh):
    std_dev = np.std(img)
    edges = cv2.Canny(img, edge_thresh // 2, edge_thresh)
    edge_density = np.sum(edges > 0) / img.size
    return std_dev > std_thresh and edge_density > 0.02

# === Load data and actual labels from filenames ===
folder = "images/demo5"
images = []
actual_labels = []
filenames = []

for filename in os.listdir(folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        file_path = os.path.join(folder, filename)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        images.append(img)
        actual_labels.append(1 if "true" in filename else 0)
        filenames.append(filename)

# === Try various combinations of thresholds ===
best_acc = 0
best_params = (0, 0)
best_predictions = []

print("Trying different thresholds...")
for std_thresh in range(0, 1000, 5):
    for edge_thresh in range(0, 2000, 10):
        predicted_labels = [
            1 if is_occupied(img, std_thresh, edge_thresh) else 0
            for img in images
        ]
        acc = accuracy_score(actual_labels, predicted_labels)
        
        if acc > best_acc:
            best_acc = acc
            best_params = (std_thresh, edge_thresh)
            best_predictions = predicted_labels.copy()

# === Output best result summary ===
print(f"\n✅ Best Accuracy: {best_acc:.4f}")
print(f"Optimal std_thresh: {best_params[0]}, edge_thresh: {best_params[1]}")
print("\nStatus\t\tActual\tPredicted\tFilename")

for actual, predicted, fname in zip(actual_labels, best_predictions, filenames):
    status = "Correct" if actual == predicted else "Wrong"
    print(f"{status}\t\t{actual}\t{predicted}\t\t{fname}")



Trying different thresholds...

✅ Best Accuracy: 0.8906
Optimal std_thresh: 40, edge_thresh: 0

Status		Actual	Predicted	Filename
Correct		1	1		box-3-true.jpg
Correct		1	1		box-11-true.jpg
Correct		0	0		box-46.jpg
Wrong		1	0		box-61-true.jpg
Correct		0	0		box-31.jpg
Correct		0	0		box-21.jpg
Correct		0	0		box-20.jpg
Correct		0	0		box-16.jpg
Correct		0	0		box-29.jpg
Wrong		1	0		box-54-true.jpg
Correct		1	1		box-58-true.jpg
Correct		1	1		box-51-true.jpg
Correct		1	1		box-9-true.jpg
Correct		1	1		box-12-true.jpg
Correct		0	0		box-33.jpg
Correct		0	0		box-41.jpg
Correct		1	1		box-10-true.jpg
Correct		0	0		box-26.jpg
Correct		1	1		box-5-true.jpg
Correct		0	0		box-43.jpg
Correct		1	1		box-2-true.jpg
Correct		0	0		box-39.jpg
Wrong		1	0		box-63-true.jpg
Correct		1	1		box-15-true.jpg
Correct		1	1		box-55-true.jpg
Correct		0	0		box-32.jpg
Correct		0	0		box-42.jpg
Correct		1	1		box-7-true.jpg
Correct		1	1		box-53-true.jpg
Correct		1	1		box-13-true.jpg
Correct		1	1		box-56-true.jpg
Correct		1	1		bo

In [19]:
TP = 0 # True Positives: Actual 1 and Pred 1
FP = 0 # False Positives: Actual 0 but Pred 1
FN = 0 # False Negatives: Actual 1 but Pred 0
TN = 0 # True Negatives: Actual 0 and Pred 0

In [20]:
count = 0
for i in actual_labels:
    count+=1 if i==1 else 0

print(count)

32


In [21]:
len(actual_labels)

64

In [22]:
count = 0
# for i in predicted_labels:
for i in best_predictions:
    count+=1 if i==1 else 0

print(count)

29


In [23]:
for i in range(64):
    TP+=1 if actual_labels[i]==1 and best_predictions[i]==1 else 0
    FP+=1 if actual_labels[i]==0 and best_predictions[i]==1 else 0
    FN+=1 if actual_labels[i]==1 and best_predictions[i]==0 else 0
    TN+=1 if actual_labels[i]==0 and best_predictions[i]==0 else 0

In [24]:
from sklearn.metrics import accuracy_score
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
accuracy_score(actual_labels, best_predictions)

0.890625

In [25]:
print(f"Accuracy = {(TP+TN)/(TP+TN+FN+FP)}")

Accuracy = 0.890625


In [ ]:
# model = RandomForestClassifier()

In [18]:
od.is_occupied(cv2.imread("images/demo/box-10-true.jpg",cv2.IMREAD_GRAYSCALE))

Std_dev = 40.523988249462306
edge_density = 0.10067637877211238


True

In [20]:
od.is_occupied(cv2.imread("images/demo/box-17.jpg",cv2.IMREAD_GRAYSCALE))

Std_dev = 41.5222200709579
edge_density = 0.04890738813735692


True

In [39]:
def is_occupied(square_img, std_thresh=27, edge_thresh=100):
    std_dev = np.std(square_img)
    edges = cv2.Canny(square_img, 50, 150)
    edge_density = np.sum(edges > 0) / square_img.size
    print(f"Std_dev = {std_dev}")
    print(f"edge_density = {edge_density}")
    return std_dev > std_thresh and edge_density > 0.02

In [40]:
is_occupied(cv2.imread("images/box-2.jpg",cv2.IMREAD_GRAYSCALE))

Std_dev = 26.26463519151476
edge_density = 0.02423411065386374


False

In [41]:
is_occupied(cv2.imread("images/box-9-true.jpg",cv2.IMREAD_GRAYSCALE))

Std_dev = 61.01861777268127
edge_density = 0.07133058984910837


True

In [42]:
count = 0
for i in predicted_labels:
    count+=1 if i==1 else 0

print(count)

45


In [32]:
folder = "../images"
actual_labels, predicted_labels,sd,edge_densities = [],[],[],[]
std_thresh=27;edge_thresh=100

file_paths=[]
print("Status\t\tActual\tPredicted")

for filename in os.listdir(folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        file_path = os.path.join(folder, filename)
        file_paths+=[file_path]
        
        square_img = cv2.imread(file_path,cv2.IMREAD_GRAYSCALE)
        square_img = square_img[10:-10,10:-10]
        
        actual_labels+=[1] if "true" in filename else [0]
        
        std_dev = np.std(square_img)
        edges = cv2.Canny(square_img, 50, 150)
        edge_density = np.sum(edges > 0) / square_img.size

        predicted_labels+=[1] if (edge_density > 0.02) else [0]
        
        sd+=[std_dev]
        edge_densities+=[edge_density]

Status		Actual	Predicted


In [33]:
df = dict({
    "file_path":file_paths,
    # "std_dev":sd,
    "edge_density":edge_densities,
    "actual_label":actual_labels,
    "predicted_label":predicted_labels
})

In [34]:
df = pd.DataFrame(df)
df

,file_path,edge_density,actual_label,predicted_label
0,../images/box-53-true.jpg,0.018006,1,0
1,../images/box-27.jpg,0.000000,0,0
2,../images/box-32.jpg,0.000000,0,0
3,../images/box-48.jpg,0.000000,0,0
4,../images/box-41-true.jpg,0.118248,1,1
...,...,...,...,...
59,../images/box-29-true.jpg,0.077130,1,1
60,../images/box-12-true.jpg,0.026068,1,1
61,../images/box-23.jpg,0.004569,0,0
62,../images/box-18.jpg,0.000000,0,0


In [35]:
df[df['actual_label']==1]

,file_path,edge_density,actual_label,predicted_label
0,../images/box-53-true.jpg,0.018006,1,0
4,../images/box-41-true.jpg,0.118248,1,1
15,../images/box-14-true.jpg,0.083042,1,1
16,../images/box-52-true.jpg,0.048643,1,1
22,../images/box-26-true.jpg,0.101317,1,1
24,../images/box-59-true.jpg,0.046762,1,1
28,../images/box-9-true.jpg,0.069874,1,1
29,../images/box-47-true.jpg,0.071486,1,1
30,../images/box-17-true.jpg,0.123085,1,1
35,../images/box-28-true.jpg,0.105617,1,1


In [36]:
df[df['actual_label']==1].shape

(16, 4)

In [37]:
df[df['predicted_label']==1]

,file_path,edge_density,actual_label,predicted_label
4,../images/box-41-true.jpg,0.118248,1,1
11,../images/box-55.jpg,0.025800,0,1
15,../images/box-14-true.jpg,0.083042,1,1
16,../images/box-52-true.jpg,0.048643,1,1
22,../images/box-26-true.jpg,0.101317,1,1
24,../images/box-59-true.jpg,0.046762,1,1
28,../images/box-9-true.jpg,0.069874,1,1
29,../images/box-47-true.jpg,0.071486,1,1
30,../images/box-17-true.jpg,0.123085,1,1
35,../images/box-28-true.jpg,0.105617,1,1


In [38]:
df[df['predicted_label']==1].shape

(18, 4)

In [96]:
sd_ones = df[df['actual_label']==1]['std_dev']

In [97]:
sd_ones.max()

71.09602774283795

In [98]:
sd_ones.min()

9.642308632540242

In [99]:
sd_zeros = df[df['actual_label']==0]['std_dev']

In [100]:
sd_zeros.max()

71.82590327375168

In [101]:
sd_zeros.min()

2.4293311013630015

In [102]:
sd_zeros

1     16.080124
2      6.351244
3      5.476936
5      4.261095
6      4.914777
7      3.661628
8      5.762281
9      4.986844
10     7.837668
11     8.951473
12     3.579955
13     3.543139
14     3.734300
17     4.557097
18     5.514952
19     4.589051
20    11.907461
21     5.885554
23     8.305973
25     4.269640
26     4.534686
27     3.810030
31     3.963837
32     5.449059
33    12.620618
34    18.140195
36     5.768385
37     5.084598
38     4.506719
39     2.429331
40    10.310428
41    16.533430
42     4.917268
43     6.424262
45     4.401940
46     6.972280
47     9.888326
49     4.339412
50    15.279067
51     3.896219
52     9.656914
53     2.700818
54     5.427652
55    71.825903
57     4.318101
58     3.986411
59     4.946881
62     3.809193
63    12.698433
Name: std_dev, dtype: float64